In [8]:
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("FRED_API_KEY")
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"


ENERGY_SERIES = {

    # Natural Gas
    "US_Regular_Conventional_Gas_price": "GASREGCOVW",
    "Natural_Gas_Exports_Index": "IQ112",
    "Natural_Gas_Imports_Index": "IR10110",
    "Natural_Gas_Consumption": "NATURALGASD11",

    # Petroleum & Crude Oil
    "Crude_Import_price Index": "IR10000",
    "CBOE_Crude_Oil_ETF_Volatility_Index": "OVXCLS",
    "WTI": "DCOILWTICO",
    "Brent": "DCOILBRENTEU",
    
    # Overall Energy Sector
    "Producer_Price_Index_Gasoline": "WPS0571",
    "Producer_Price_Index_Final_Demand_Energy": "PPIDES",
    "Avg_Gasoline_Unleaded_Regular": "APU000074714"
}

def fetch_fred_series(series_id, start_date="2015-01-01"):
    params = {
        "series_id": series_id,
        "api_key": API_KEY,
        "file_type": "json",
        "observation_start": start_date
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch {series_id}: {response.status_code}")
        return pd.DataFrame()

    data = response.json()
    if "observations" not in data:
        print(f"No observations found for {series_id}")
        return pd.DataFrame()

    df = pd.DataFrame(data["observations"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df["date"] = pd.to_datetime(df["date"])
    df = df[["date", "value"]].set_index("date")
    df.columns = [series_id]
    return df

#fetch
all_energy_series = []

for label, series_id in ENERGY_SERIES.items():
    print(f"Fetching {label} ({series_id})")
    df = fetch_fred_series(series_id)
    if not df.empty:
        df.columns = [label]
        all_energy_series.append(df)

print(all_energy_series)

Fetching US_Regular_Conventional_Gas_price (GASREGCOVW)
Fetching Natural_Gas_Exports_Index (IQ112)
Fetching Natural_Gas_Imports_Index (IR10110)
Fetching Natural_Gas_Consumption (NATURALGASD11)
Fetching Crude_Import_price Index (IR10000)
Fetching CBOE_Crude_Oil_ETF_Volatility_Index (OVXCLS)
Fetching WTI (DCOILWTICO)
Fetching Brent (DCOILBRENTEU)
Fetching Producer_Price_Index_Gasoline (WPS0571)
Fetching Producer_Price_Index_Final_Demand_Energy (PPIDES)
Fetching Avg_Gasoline_Unleaded_Regular (APU000074714)
[            US_Regular_Conventional_Gas_price
date                                         
2015-01-05                              2.136
2015-01-12                              2.066
2015-01-19                              1.999
2015-01-26                              1.982
2015-02-02                              2.013
...                                       ...
2025-03-24                              2.999
2025-03-31                              3.040
2025-04-07                    